# Clickbait Detection: Logistic Regression (No Scaling) & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook αναπτύσσουμε και αξιολογούμε ένα μοντέλο **Logistic Regression** χωρίς την εφαρμογή κανονικοποίησης (Scaling).

### Γιατί "No Scaling";
Παρόλο που η Logistic Regression συνήθως απαιτεί Scaling, τα πειραματικά δεδομένα έδειξαν ότι στα **UMAP Embeddings**, η εφαρμογή `StandardScaler` καταστρέφει τη γεωμετρική πληροφορία (πυκνότητα/αποστάσεις).
Επομένως, εδώ δοκιμάζουμε την εκπαίδευση στα "ωμά" δεδομένα του UMAP, αναμένοντας υψηλότερη επίδοση.

## Στόχοι
1. **Φόρτωση Δεδομένων**: Χρήση των UMAP-500 parquets (χωρίς περαιτέρω επεξεργασία).
2. **Βελτιστοποίηση (Optuna)**: Εύρεση παραμέτρων (`C`, `penalty`, `solver`) απευθείας στα raw embeddings.
3. **Champion Model**: Εκπαίδευση του τελικού μοντέλου στο σύνολο Train+Valid.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Απόκρυψη FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# Ρύθμιση Paths
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## Φόρτωση Δεδομένων

Φορτώνουμε τα Train/Valid/Test sets.
**Σημείωση:** Δεν εφαρμόζουμε καμία μέθοδο Scaling. Τα δεδομένα εισάγονται στο μοντέλο ακριβώς όπως παρήχθησαν από το UMAP.

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"⏳ Έναρξη φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features
        feature_cols = [c for c in df.columns if c.startswith("umap_")]
        if not feature_cols:
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 2. Εντοπισμός Label
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        if label_col is None:
             raise ValueError(f"Δεν βρέθηκε label στήλη στο {filename}")

        if label_col in feature_cols:
            feature_cols.remove(label_col)

        y = df[label_col].values.astype(int)
        X = df[feature_cols].values.astype(np.float32)

        loaded_data[name] = (X, y)
        print(f"   ✅ [{name}] Loaded: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## Βελτιστοποίηση Υπερπαραμέτρων (Tuning)

Ορίζουμε τη συνάρτηση `objective`.
* Χρησιμοποιούμε τον solver `saga` που υποστηρίζει όλα τα penalties (l1, l2, elasticnet).
* Αυξάνουμε το `max_iter` στο **2000** για να βοηθήσουμε τη σύγκλιση, καθώς η έλλειψη scaling μπορεί να κάνει τη βελτιστοποίηση πιο αργή.
* **Δεν χρησιμοποιούμε StandardScaler.**

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # --- 1. Hyperparameters Space ---
    # Ο 'saga' είναι ο πιο ευέλικτος solver για μεγάλα datasets
    solver = "saga"
    penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet"])
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)

    # Αυξάνουμε τα iterations γιατί χωρίς scaling η σύγκλιση είναι πιο δύσκολη
    max_iter = 2000

    # --- 2. Model Setup ---
    params = dict(
        solver=solver,
        penalty=penalty,
        C=C,
        max_iter=max_iter,
        random_state=42,
        n_jobs=-1
    )

    if penalty == "elasticnet":
        params["l1_ratio"] = trial.suggest_float("l1_ratio", 0.0, 1.0)

    model = LogisticRegression(**params)

    # --- 3. Training & Eval (NO SCALING) ---
    t0 = time.time()
    model.fit(X_tr, y_tr)
    train_time = time.time() - t0

    preds = model.predict(X_v)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    metrics = {"val_f1": f1, "val_accuracy": acc, "training_time_sec": training_time}

    # Log param: No Scaling
    params_to_log = params.copy()
    params_to_log["scaler"] = "None"

    mlflow_helper.log_optuna_trial(trial, params_to_log, metrics, model, run_name_prefix="LR_Trial")

    return f1

## Εκτέλεση Πειράματος & Champion Model

1. **Tuning:** 20 trials για εύρεση του βέλτιστου συνδυασμού (Regularization strength `C`, Penalty type).
2. **Champion Model:** Εκπαίδευση στο πλήρες dataset (Train + Valid) χωρίς scaling.
3. **Evaluation:** Αξιολόγηση στο Test set.

In [ ]:
EXPERIMENT_NAME = "Clickbait_LR_UMAP_NoScaling"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\n🚀 Έναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Tuning ---
print("\n🔍 [ΦΑΣΗ 1] Hyperparameter Tuning...")
with mlflow.start_run(run_name="LR_Hyperparameter_Tuning"):
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("scaling", "None") # Explicitly log No Scaling

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler)

    study.optimize(
        lambda t: objective(t, X_train, y_train, X_val, y_val),
        n_trials=20
    )

    print("🏆 Best F1:", study.best_value)
    print("🏆 Best Params:", study.best_params)


# --- ΦΑΣΗ 2: Champion Model ---
print("\n👑 [ΦΑΣΗ 2] Champion Model Training...")
with mlflow.start_run(run_name="LR_Champion_Model") as final_run:
    best = study.best_params

    # 1. Προετοιμασία Δεδομένων (Full Train - No Scaling)
    X_full = np.vstack([X_train, X_val])
    y_full = np.concatenate([y_train, y_val])

    # 2. Setup Model
    champion_params = dict(
        solver="saga",
        penalty=best["penalty"],
        C=best["C"],
        max_iter=2000,
        random_state=42,
        n_jobs=-1
    )
    if best["penalty"] == "elasticnet":
        champion_params["l1_ratio"] = best["l1_ratio"]

    final_model = LogisticRegression(**champion_params)

    # 3. Training
    t0 = time.time()
    final_model.fit(X_full, y_full)
    final_train_time = time.time() - t0

    # 4. Logging
    mlflow.log_params(champion_params)
    mlflow.log_param("scaler", "None")
    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # 5. Evaluation
    print("📈 Αξιολόγηση στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test, # Raw UMAP data
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\n✅ Ολοκληρώθηκε. Run ID: {final_run.info.run_id}")